# pix2pix

## Load images

## download dataset
The used dataset is the [nyu_depth Dataset](https://cs.nyu.edu/~silberman/datasets/nyu_depth_v2.html) which contains images, dense depthmaps and raw depthmaps.

To show progress bars in Jupyter Lab run install the jupyterlab-manager widget before training:
``` bash
$ jupyter labextension install @jupyter-widgets/jupyterlab-manager
```

In [ ]:
import os

path = "datasets/nyu_depth.mat"

if not os.path.isfile(path):
    ! wget -O datasets/nyu_depth.mat http://horatio.cs.nyu.edu/mit/silberman/nyu_depth_v2/nyu_depth_v2_labeled.mat

### common dependencies

In [ ]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import csv
import random
import math
from PIL import Image
from tqdm import tqdm
import scipy.io
import h5py
from keras_tqdm import TQDMNotebookCallback

## load dataset

In [ ]:
dataset = h5py.File(path,'r')

print('loading raw depths...')
xs = np.array(dataset.get('rawDepths'))
xs = xs.reshape((xs.shape[0], 1, xs.shape[1], xs.shape[2]))
xs = np.swapaxes(xs, 1, 3)
x_center = math.floor((xs.shape[1] - 256) / 2)
y_center = math.floor((xs.shape[2] - 256) / 2)
xs = xs[:,x_center:x_center+256,y_center:y_center+256,:]
print('changing range...')
xs = xs / 255

print('loading processed depths...')
ys = np.array(dataset.get('depths'))
ys = ys.reshape((ys.shape[0], 1, ys.shape[1], ys.shape[2]))
ys = np.swapaxes(ys, 1, 3)
ys = ys[:,x_center:x_center+256,y_center:y_center+256,:]
print('changing range...')
ys = ys / 255

#### Visualization of random images and their labels

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(6,6,figsize=(16,16))
fig.tight_layout()
ax = ax.flatten()

for i in range(18):
    rand = np.random.randint(len(xs)-1)
    x = xs[rand].reshape((xs[rand].shape[0], xs[rand].shape[1]))
    y = ys[rand].reshape((ys[rand].shape[0], ys[rand].shape[1]))
    
    ax[2 * i].imshow(x)
    ax[2 * i].set_title(f"{i}_x")
    ax[2 * i].axis("off")
    ax[2 * i + 1].imshow(y)
    ax[2 * i + 1].set_title(f"{i}_y")
    ax[2 * i + 1].axis("off")

### import pix2pix and generate model

In [ ]:
%run pix2pix.ipynb

model = Pix2pix(output_dim=1)

In [ ]:
((train_x, train_y), (test_x, test_y)) = model.split_dataset(xs, ys, validation_split=0.05)

## Checkpoint stuff

In [ ]:
checkpoint_dir = 'checkpoints/sparse_to_dense_depthmap/'
checkpoint_path = checkpoint_dir + 'checkpoint-{epoch:04d}.ckpt'
os.makedirs(checkpoint_dir, exist_ok=True)

# if a checkpoint exists => build model and load weights
if tf.train.latest_checkpoint(checkpoint_dir) != None:
    model.build(train_x.shape)
    status = model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

### fit model

In [ ]:
tqdm_callback = TQDMNotebookCallback(inner_description_update="Epoch: {epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=False)

model.fit(train_x, train_y, batch_size=10, epochs=100, validation_data=(test_x, test_y), callbacks=[tqdm_callback, checkpoint_callback])

### visualize results of test data

In [ ]:
out = model.predict(test_x, batch_size=10)
for i in range(len(out)):
    fig, ax = plt.subplots(1,3,figsize=(10,10))
    fig.tight_layout()
    ax = ax.flatten()

    x = test_x[i].reshape((256, 256))
    y = test_y[i].reshape((256, 256))
    o = out[i].reshape((256, 256))
    
    ax[0].imshow(x)
    ax[0].set_title("x")
    ax[0].axis("off")
    ax[1].imshow(y)
    ax[1].set_title("y")
    ax[1].axis("off")
    ax[2].imshow(o)
    ax[2].set_title("g(x)")
    ax[2].axis("off")